In [1]:
import torch
import numpy as np
import json
import os
import cv2
import random
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

import tqdm

BLUE=(255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)
YELLOW = (0, 255, 255)
PINK = (255, 0, 255)
BLACK = (0, 0, 0)
ORANGE = (0, 127, 255)
CUSTOM = (255,170,170)
COLOR_CLASS = {0: BLUE, 1:GREEN, 2:RED, 3:YELLOW, 4:PINK, 5:BLACK, 6:ORANGE, 7:CUSTOM}

def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def cocoToAbsoluteBox(cocoBox):
    #xywh -> xyxy
    return [cocoBox[0], cocoBox[1], cocoBox[0]+cocoBox[2], cocoBox[1]+cocoBox[3]]


# matplotlib 
def plot(key, list_bbox_):
    fig = plt.figure(figsize=(15,10))

    plt.hist(list_bbox_[key], color = 'blue', edgecolor = 'black',
            bins = int(len(set(list_bbox_[key]))))

    # Add labels
    plt.title('Histogram of {}'.format(key))
    plt.xlabel(key)
    plt.ylabel('count')
#     plt.savefig("d/{}_distribution.png".format(key))

In [2]:
df = pd.read_csv('/home/hana/sonnh/kaggle-vin/dataset/original_data/train.csv')

In [3]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN


In [26]:
df = pd.read_csv('/home/hana/sonnh/kaggle-vin/dataset/images_only/train_only_box.csv')
# df = df[df['fold'] == 0]

In [11]:
def get_shape(image_name):
    try:
        image = cv2.imread('{}/{}'.format('/home/hana/sonnh/kaggle-vin/dataset/images_only/train', image_name))
    
        return image_name, image.shape[0], image.shape[1]
    except:
        return image_name, 1, 1


import os
from multiprocessing import Pool

pool = Pool()
data = pool.map(get_shape, os.listdir('/home/hana/sonnh/kaggle-vin/dataset/images_only/train'))

data_ = {}
for i in range(len(data)):
    data_[data[i][0]] = {'height': data[i][1], 'width':data[i][2]}

In [27]:
data = {}

for i in range(len(df)):
    image_id = df.iloc[i]['image_id']
    class_id = df.iloc[i]['class_id']
    if image_id not in data:
        data[image_id] = {'bbox':[], 'class_id':[]}
        
  
    xmax = df.iloc[i]['x_max']
    ymax = df.iloc[i]['y_max']
    xmin = df.iloc[i]['x_min']
    ymin = df.iloc[i]['y_min']
    
    data[image_id]['bbox'].append([xmin, ymin, xmax, ymax])
    data[image_id]['class_id'].append(class_id)

In [34]:
count = 0
iou = 0.6
for image_id in data:
    for i in range(len(data[image_id])):
        for j in range(i+1, len(data[image_id])):
            if bb_intersection_over_union(data[image_id]['bbox'][i], data[image_id]['bbox'][j]) > iou /
            and data[image_id]['class_id'][i] == data[image_id]['class_id'][j]:
                count += 1
#                 print(data[image_id][i], data[image_id][j])
#     print(count)
count

821

In [29]:
len(df)

36096